# Feed Forward Neural Network on Fashion MNIST
---
Don't forget to use **https://pytorch.org/docs/stable/**

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

---

## Prepare Fashion MNIST dataset
We want to preprocess training data, specifically to have flatten shape `(28, 28) -> 784` in `torch.Tensor` format.

In [ ]:
import torch
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Compose
from torch.utils.data import DataLoader

In [ ]:
class ReshapeTransform:
    def __init__(self, new_size=(-1,)):
        self.new_size = new_size

    def __call__(self, sample):
        return torch.reshape(sample, self.new_size)

In [ ]:
transformations = Compose([ToTensor(), ReshapeTransform()])

In [ ]:
train_dataset = FashionMNIST('./dataset_fashion_mnist/', download=True, train=True, 
                             transform=transformations, 
                             target_transform=None)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

valid_dataset = FashionMNIST('./dataset_fashion_mnist/', download=True, train=False, 
                             transform=transformations, 
                             target_transform=None)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [ ]:
train_dataset[0]

In [ ]:
next(iter(train_loader))

---

## Define feed forward neural network
In case we use `torch.nn` modules, we don't need to register tensor with `torch.nn.Parameter`.   

**Important:** Don't forget to setup `.eval()` or `.train()` modes for model to enforce proper behaviour of certain layers as `torch.nn.Dropout` or `torch.nn.BatchNorm1d`.

### Architecture

In [ ]:
from torch.nn import Module
from torch.nn import ReLU, Tanh, Dropout, Softmax, Linear, BatchNorm1d
from torch.nn import MSELoss, CrossEntropyLoss, NLLLoss
from torch.optim import Adam, SGD
from torch.nn.init import xavier_uniform_, normal_

In [ ]:
class FeedForwardNeuralNet(torch.nn.Module):
    def __init__(self):
        super(FeedForwardNeuralNet, self).__init__()
        
        self.layer_1 = Linear(784, 10)
        ##########################
        # TODO: Add extra layer. #
        ##########################
        
        ##########################################################
        # TODO: Prepare batch norlmalization and dropout module. #
        ##########################################################
        
    def forward(self, input_batch):
        prediction = self.layer_1(input_batch)
        ###################################################
        # TODO: Stack activation -> bn -> dropout layers. #
        ###################################################
        
        return torch.softmax(prediction, dim=1)
        ############################################################################################
        # TODO: Numeric optimization                                                               #
        #       Switch torch.softmax -> torch.log_softmax during training. Softmax leave for eval. #                               #
        #       Use torch.nn.NLLLoss as loss (https://pytorch.org/docs/stable/nn.html#nllloss).    #
        #       Why is it cool?                                                                    #
        ############################################################################################

In [ ]:
feed_forward_neural_net = FeedForwardNeuralNet()

In [ ]:
feed_forward_neural_net

In [ ]:
feed_forward_neural_net.state_dict()

In [ ]:
images, labels = next(iter(valid_loader))

In [ ]:
feed_forward_neural_net.eval()
predictions = feed_forward_neural_net(images)
feed_forward_neural_net.train()
predictions[:4]

### Optimizers and loss function

In [ ]:
#######################################
# TODO: Try NLLLoss with log_softmax. #
#######################################
loss_fce = CrossEntropyLoss()
loss_fce

In [ ]:
loss_fce(predictions, labels)

In [ ]:
###############################
# TODO: Adjust learning rate. #
###############################
optimizer = SGD(feed_forward_neural_net.parameters(), lr=0.5)

###################################
# TODO: Switch optimizer to Adam. #
###################################
optimizer

### Training of neural net

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_valid_acc_and_loss(model, loss_fce, valid_loader):
    accuracy = 0
    loss = 0
    was_training = model.training
    
    model.eval()
    for images, labels in valid_loader:
        predictions = model(images)
        accuracy += (predictions.argmax(dim=1) == labels).type(torch.FloatTensor).mean().item() 
        loss += loss_fce(predictions, labels).item()
    model.train(mode=was_training)
    return accuracy / len(valid_loader) * 100, loss / len(valid_loader)

In [ ]:
get_valid_acc_and_loss(feed_forward_neural_net, loss_fce, valid_loader)

In [ ]:
from collections import deque

# Initial params setup.
epochs = 2
report_period = 100
batch_iteration = 0

# Storing of some data.
train_leak_loss = deque(maxlen=report_period)
train_loss_history = []
valid_loss_history = []
valid_acc_history = []

In [ ]:
for epoch in range(epochs):
    # Setup net to train mode and go through one epoch.
    feed_forward_neural_net.train()
    for images, labels in train_loader:
        batch_iteration += 1
        
        ##################
        # Training Phase #
        ##################
        optimizer.zero_grad()
        predictions = feed_forward_neural_net.forward(images)
        loss = loss_fce(predictions, labels)
        loss.backward()
        optimizer.step()
        
        
        ####################
        # Validation Phase #
        ####################
        train_leak_loss.append(loss.item())
        if batch_iteration % report_period == 0:
            feed_forward_neural_net.eval()
            
            # We don't want to collect info for gradients from here.
            with torch.no_grad():
                valid_accuracy, valid_loss = get_valid_acc_and_loss(feed_forward_neural_net, loss_fce, valid_loader)
                
            print(f"Epoch: {epoch+1}/{epochs}.. ",
                  f"Train Loss: {round(np.mean(train_leak_loss), 2)}.. ",
                  f"Valid Loss: {round(valid_loss, 2)}.. ",
                  f"Valid Acc: {round(valid_accuracy, 2)}%")
            
            train_loss_history.append(np.mean(train_leak_loss))
            valid_loss_history.append(valid_loss)
            valid_acc_history.append(valid_accuracy)
                   
            feed_forward_neural_net.train()

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
ax.set_xlabel('Iteration')
ax.set_ylabel('Cross Entropy')
plt.plot(train_loss_history, label='Train loss')
plt.plot(valid_loss_history, label='Valid loss')
plt.legend(frameon=False)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.plot(valid_acc_history, label='Valid acc')
ax = plt.gca()
ax.set_xlabel('Iteration')
ax.set_ylabel('Acc(%)')
plt.legend(frameon=False)

---

## Results evaluation

In [ ]:
feed_forward_neural_net.eval()

### View single images and predictions

In [ ]:
from image_processing_workshop.visual import plot_classify, plot_image

In [ ]:
plot_classify(input_tensor=valid_dataset[12][0], 
              model=feed_forward_neural_net, image_shape=[28,28])

### Load reuslts to pandas df

In [ ]:
from image_processing_workshop.eval import get_results_df
from image_processing_workshop.visual import plot_df_examples

In [ ]:
df = get_results_df(feed_forward_neural_net, valid_loader)
df.head(10)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
ax.set_xlabel('Prediction Score')
df[df.label_class_name=='Dress'].label_class_score.hist(ax=ax)

In [ ]:
plot_df_examples(df.iloc[:25], image_shape=[28, 28])

### Precision

In [ ]:
from image_processing_workshop.eval import get_precision

In [ ]:
get_precision(df, 'Dress')

### Recall

In [ ]:
from image_processing_workshop.eval import get_recall

In [ ]:
get_recall(df, 'Dress')

### Overall Recall and Precision

In [ ]:
from image_processing_workshop.eval import get_rec_prec

In [ ]:
get_rec_prec(df)

### Accuracy

In [ ]:
from image_processing_workshop.eval import get_accuracy

In [ ]:
get_accuracy(df)

### False Positives

In [ ]:
from image_processing_workshop.eval import get_false_positives

In [ ]:
fp = get_false_positives(df, label_class_name='Shirt')

In [ ]:
plot_df_examples(fp, image_shape=[28, 28])

In [ ]:
fp = get_false_positives(df, label_class_name='Shirt', predicted_class_name='Pullover')

In [ ]:
plot_df_examples(fp, image_shape=[28, 28])

### Confusion Matrix

In [ ]:
from image_processing_workshop.visual import plot_coocurance_matrix

In [ ]:
plot_coocurance_matrix(df, use_log=False)